In [22]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer


In [12]:
load_dotenv(find_dotenv(), override=True)

True

In [3]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [ ]:
pc.list_indexes()

In [35]:
index = pc.Index(os.environ["PINECONE_INDEX_HOST"])

# Semantic Search

In [17]:
df = pd.read_csv("course_descriptions.csv", encoding='cp1252')

In [18]:
df.head()

,course_name,course_slug,course_technology,course_description,course_topic,course_description_short
0,Introduction to Tableau,tableau,tableau,Tableau is now one of the most popular busines...,data visualization,Teaching you how to tell compelling stories wi...
1,The Complete Data Visualization Course with Py...,data-visualization,python,The Data Visualization course is designed for ...,data visualization,Teaching you how to master the art of creating...
2,Introduction to R Programming,introduction-to-r-programming,r,R is one of the best programming languages spe...,programming,"Providing you with the skills to manipulate, a..."
3,Data Preprocessing with NumPy,data-preprocessing-numpy,python,This course is designed to show you how to wor...,data processing,This course will guide you through one of Pyth...
4,Introduction to Data and Data Science,intro-to-data-and-data-science,theory,Working with data is an essential part of main...,machine learning,Introducing you to the field of data science a...


In [19]:
def create_description(row) -> str:
    return f""" The course name is {row['course_name']}, The course slug is {row['course_slug']}, The course description is {row['course_description']} """

In [20]:
df['description_new'] = df.apply(create_description, axis = 1)

In [21]:
pd.set_option('display.max_rows', 106)
df['description_new']

0       The course name is Introduction to Tableau, T...
1       The course name is The Complete Data Visualiz...
2       The course name is Introduction to R Programm...
3       The course name is Data Preprocessing with Nu...
4       The course name is Introduction to Data and D...
5       The course name is Data Cleaning and Preproce...
6       The course name is Introduction to Business A...
7       The course name is Data Analysis with Excel P...
8       The course name is SQL, The course slug is sq...
9       The course name is Credit Risk Modeling in Py...
10      The course name is Python Programmer Bootcamp...
11      The course name is SQL + Tableau + Python, Th...
12      The course name is Introduction to Jupyter, T...
13      The course name is Statistics, The course slu...
14      The course name is Mathematics, The course sl...
15      The course name is Introduction to Excel, The...
16      The course name is Probability, The course sl...
17      The course name is Star

In [23]:
# create embeddings

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Ashish Biswal\AI Projects\langchain-rag-nlp-toolkit\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ashish Biswal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
def create_embeddings(row):
    combined_text = ' '.join([str(row[i]) for i in ['description_new', 'course_description', 'course_description_short']])
    encoding = model.encode(combined_text, show_progress_bar = True)
    return encoding

In [26]:
df['embedding'] = df.apply(create_embeddings, axis = 1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
df['embedding']

0      [0.0053340876, -0.022557598, -0.081612445, -0....
1      [0.045281474, -0.019791452, -0.026416214, -0.0...
2      [-0.05802464, 0.0027213984, -0.088361815, 0.02...
3      [-0.06035171, -0.014297017, -0.041090786, -0.0...
4      [-0.0077002076, 5.1757746e-07, -0.0064406553, ...
5      [-0.03915095, -0.0076299096, -0.045978986, -0....
6      [-0.052439023, -0.00077925454, -0.098035656, 0...
7      [-0.017170833, 0.036006987, -0.08626053, -0.02...
8      [0.0072859433, -0.023177236, -0.06795352, 0.06...
9      [-0.02885249, -0.04784123, -0.05688989, 0.0102...
10     [-0.0685995, -0.020440752, -0.024613503, 0.002...
11     [-0.00033983338, -0.07239232, -0.088360794, 0....
12     [-0.031387053, -0.019974135, -0.045017302, -0....
13     [-0.011427382, -0.011978196, -0.04836095, 0.06...
14     [-0.030334448, -0.03904209, 0.012471507, -0.05...
15     [-0.031239036, 0.014678238, -0.09107833, -0.00...
16     [-0.032531366, -0.033825744, -0.007306792, -0....
17     [-0.11161639, -0.0039710

In [31]:
vectors_to_upsert = [(str(row["course_name"]), row["embedding"].tolist()) for _,row in df.iterrows()]

In [36]:
index.upsert(vectors = vectors_to_upsert)

UpsertResponse(upserted_count=106, _response_info={'raw_headers': {'date': 'Fri, 16 Jan 2026 07:23:35 GMT', 'content-type': 'application/json', 'content-length': '21', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '165995', 'x-pinecone-request-latency-ms': '1621', 'x-pinecone-request-id': '3622925496434879484', 'x-envoy-upstream-service-time': '255', 'x-pinecone-response-duration-ms': '1624', 'grpc-status': '0', 'server': 'envoy'}})

# Quering

In [37]:
query = "Clustering"
query_embedding = model.encode(query)

In [38]:
query_results = index.query(
    vector = query_embedding.tolist(),
    top_k = 5,
    include_values = True
)

query_results

QueryResponse(matches=[{'id': 'Machine Learning with K-Nearest Neighbors',
 'score': 0.326045513,
 'values': [-0.125032037,
            -0.0881662,
            0.00857302081,
            -0.0610800758,
            0.00338004157,
            -0.0436201245,
            -0.0342044905,
            -0.0684913099,
            -0.0572946668,
            -0.0318441875,
            -0.0593687333,
            0.0206152219,
            0.0874756873,
            -0.0199538246,
            -0.0739126429,
            -0.00586107466,
            0.0217842497,
            0.0524356179,
            -0.0544125214,
            -0.0746904835,
            0.0410010591,
            0.0336986966,
            -0.0109662274,
            0.00124627969,
            0.00869998801,
            0.00546237733,
            0.131698817,
            0.0501171835,
            -0.0270520356,
            -0.00580718974,
            -0.025094144,
            -0.0150315519,
            0.0147006381,
            0.0080079110

In [40]:
score_threshold = 0.3
for match in query_results["matches"]:
    if match['score'] >= score_threshold:
        print(f"Matched item ID: {match['id']}, score: {match['score']}")

Matched item ID: Machine Learning with K-Nearest Neighbors, score: 0.326045513
Matched item ID: Machine Learning in Excel, score: 0.306141376
